In [1]:
import pandas as pd

# Read in the data
home_data = pd.read_csv("C:\\Users\\shgupta\\OneDrive - Five9\\Documents\\Analytica\\train.csv")

# Create a new feature TotalSF
home_data['TotalSF'] = home_data['TotalBsmtSF'] + home_data['1stFlrSF'] + home_data['2ndFlrSF']

# Check the new feature
print(home_data[['TotalBsmtSF', '1stFlrSF', '2ndFlrSF', 'TotalSF']].head())


   TotalBsmtSF  1stFlrSF  2ndFlrSF  TotalSF
0          856       856       854     2566
1         1262      1262         0     2524
2          920       920       866     2706
3          756       961       756     2473
4         1145      1145      1053     3343


In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_selection import RFECV
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

# Read in the data
home_data = pd.read_csv("C:\\Users\\shgupta\\OneDrive - Five9\\Documents\\Analytica\\train.csv")

# Perform mean imputation on missing values
home_data.fillna(home_data.mean(), inplace=True)

# Create a new feature for total square footage
home_data['TotalSF'] = home_data['TotalBsmtSF'] + home_data['1stFlrSF'] + home_data['2ndFlrSF']

# Perform one-hot encoding on categorical variables
categorical_features = ['MSZoning', 'Street', 'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'Heating', 'HeatingQC', 'CentralAir', 'Electrical', 'KitchenQual', 'Functional', 'FireplaceQu', 'GarageType', 'GarageFinish', 'GarageQual', 'GarageCond', 'PavedDrive', 'PoolQC', 'Fence', 'MiscFeature', 'SaleType', 'SaleCondition']
home_data = pd.get_dummies(home_data, columns=categorical_features)

# Split the data into features and target variable
X = home_data.drop('SalePrice', axis=1)
y = home_data['SalePrice']

# Create a MinMaxScaler object
scaler = MinMaxScaler()

# Fit the scaler to the features and transform the features
X_scaled = scaler.fit_transform(X)

# Create a random forest regressor object
rf = RandomForestRegressor(random_state=42)

# Perform recursive feature elimination with cross-validation
rfecv = RFECV(estimator=rf, step=1, cv=5, scoring='neg_mean_squared_error')
rfecv.fit(X_scaled, y)

# Print the selected features
print("Optimal number of features: ", rfecv.n_features_)
print("Selected features: ", X.columns[rfecv.support_])

# Select the important features
X_important = rfecv.transform(X_scaled)

# Perform train-test split
X_train, X_test, y_train, y_test = train_test_split(X_important, y, test_size=0.3, random_state=42)

# Fit a linear regression model
lr = LinearRegression()
lr.fit(X_train, y_train)

# Evaluate the model
train_score = lr.score(X_train, y_train)
test_score = lr.score(X_test, y_test)
print("Train R^2 score: ", train_score)
print("Test R^2 score: ", test_score)
plt.show()

